In [1]:
from device import Device
from rgf import GreensFunction
import scipy as sp
from hamiltonian import Hamiltonian
from helper import Helper_functions
import scipy.sparse as spa
import numpy as np
import scipy.sparse as sp
from lead_self_energy import LeadSelfEnergy
from scipy.sparse import bmat, identity, random, csc_matrix
from scipy.sparse.linalg import eigsh, eigs, spsolve
import time
from NEGF_unit_generation import UnitCell

In [2]:
x = UnitCell(1,2, orientiation=(1,2,3,0))



In [3]:
A= np.array(np.random.rand(4,4))
A = (A + A.T) / 2
print(A)
print(np.linalg.inv(A))
print(GreensFunction.rgf_inverse(A=A, block_size=2))

[[0.91863796 0.45549008 0.52899954 0.58227958]
 [0.45549008 0.51371747 0.39978402 0.62529805]
 [0.52899954 0.39978402 0.14520727 0.65858815]
 [0.58227958 0.62529805 0.65858815 0.0117717 ]]
[[ 1.29085664 -3.15626027  1.84699547  0.47186197]
 [-3.15626027 -1.15840653  3.84768578  2.39005321]
 [ 1.84699547  3.84768578 -5.2658108  -1.13981562]
 [ 0.47186197  2.39005321 -1.13981562 -1.57854395]]
[array([[ 1.71873402+0.j, -2.07036242+0.j],
       [-2.07036242+0.j,  1.4286089 +0.j]]), array([[-1.95159902-0.j,  0.11365129+0.j],
       [ 0.11365129+0.j, -1.26322115-0.j]])]


In [4]:
import os

# --- Add these lines at the very top of your script ---
# This must be done BEFORE importing numpy or other scientific libraries.
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['VECLIB_MAXIMUM_THREADS'] = '1'
os.environ['NUMEXPR_NUM_THREADS'] = '1'

import numpy as np
import multiprocessing
import matplotlib.pyplot as plt
from collections import defaultdict
from itertools import product
import time
from device import Device
from rgf import GreensFunction
from hamiltonian import Hamiltonian

def DOS(param): 
    
    E, ky = param
    dev = Device(5e-9, 1e-9)
    
    ham = Hamiltonian(dev)

    gf = GreensFunction(dev, ham)
    lse = LeadSelfEnergy(dev, ham)

    H = ham.create_sparse_channel_hamlitonian(ky, blocks=False)

    sl = csc_matrix(lse.iterative_self_energy(E, ky, "left"))
    sr = csc_matrix(lse.iterative_self_energy(E, ky, "right"))
    block_size = sl.shape[0]
    H[:block_size, :block_size] += sl
    H[-block_size:,-block_size:] += sr

    H = csc_matrix(np.eye(H.shape[0], dtype=complex) * E) - H

    I = csc_matrix(np.eye(H.shape[0], dtype=complex))

    H.shape[0]

    A = spsolve(H,I)
    tr = -1/np.pi * np.imag(A.trace())

    return (E, ky, tr)


energy_values = np.linspace(-2, 3, 50)  # More energy points for better resolution
ky_values = np.linspace(0, 1.0, 20)     # Fewer ky points for faster computation
dk = ky_values[1] - ky_values[0]        # The differential k_y step

param_grid = list(product(energy_values, ky_values))

print(f"Starting DOS calculations for {len(param_grid)} (E, ky) pairs...")
print(f"Energy range: {energy_values[0]:.2f} to {energy_values[-1]:.2f} eV")
print(f"ky range: {ky_values[0]:.2f} to {ky_values[-1]:.2f}")

start_time = time.time()

# Use fewer processes for debugging - increase to 32 once working
with multiprocessing.Pool(processes=8) as pool:
    results = pool.map(DOS, param_grid)
    
end_time = time.time()
print(f"Calculations finished in {end_time - start_time:.2f} seconds.")

# Aggregate traces for each energy value (sum over all ky values)
energy_traces = defaultdict(complex)
valid_results = 0

for energy, ky, trace in results:
    if not (np.isnan(trace) or np.isinf(trace)):
        energy_traces[energy] += trace
        valid_results += 1
    else:
        print(f"Invalid result at E={energy:.3f}, ky={ky:.3f}: {trace}")

print(f"Valid results: {valid_results}/{len(results)}")

# Sort energies and calculate DOS
dos_energies = sorted(energy_traces.keys())

# DOS formula: D(E) = -(1/π) * Im[Tr(G_R(E))] * (dk/2π)

dos_values = []
for E in dos_energies:
    trace_val = energy_traces[E]
    dos_val = trace_val *  (dk / (2 * np.pi))
    dos_values.append(dos_val)

print(f"DOS calculation complete. Energy points: {len(dos_energies)}")
# Plotting the Density of States
plt.figure(figsize=(12, 8))

# Main DOS plot
plt.subplot(2, 1, 1)
plt.plot(dos_energies, dos_values, 'b-', linewidth=2, label='Density of States D(E)')
plt.xlabel('Energy (eV)')
plt.ylabel('DOS (states/eV)')
plt.title('Density of States from Green\'s Function')
plt.grid(True, alpha=0.3)
plt.legend()



plt.tight_layout()
plt.savefig("dos.png", dpi=300, bbox_inches='tight')
plt.show()

# Save data for further analysis
np.savetxt("dos_data.txt", np.column_stack([dos_energies, dos_values]), 
            header="Energy(eV)\tDOS(states/eV)", fmt="%.6e")

Starting DOS calculations for 1000 (E, ky) pairs...
Energy range: -2.00 to 3.00 eV
ky range: 0.00 to 1.00


/home/nandan_diwan/Desktop/NEGF_sim/.venv/lib/python3.11/site-packages/scipy/sparse/_index.py:201: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil and dok are more efficient.
  self._set_arrayXarray_sparse(i, j, x)
/home/nandan_diwan/Desktop/NEGF_sim/.venv/lib/python3.11/site-packages/scipy/sparse/_index.py:201: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil and dok are more efficient.
  self._set_arrayXarray_sparse(i, j, x)
/home/nandan_diwan/Desktop/NEGF_sim/.venv/lib/python3.11/site-packages/scipy/sparse/_index.py:201: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil and dok are more efficient.
  self._set_arrayXarray_sparse(i, j, x)
/home/nandan_diwan/Desktop/NEGF_sim/.venv/lib/python3.11/site-packages/scipy/sparse/_index.py:201: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil and dok are more efficient.
  self._

KeyboardInterrupt: 

In [ ]:
DOS((0,0))

/home/nandan_diwan/Desktop/NEGF_sim/.venv/lib/python3.11/site-packages/scipy/sparse/_index.py:201: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil and dok are more efficient.
  self._set_arrayXarray_sparse(i, j, x)


(0, 0, np.float64(48.5218780992764))

In [ ]:
G_R= GreensFunction.rgf_inverse(A=H,block_size= block_size)

In [ ]:
G_R

[array([[-1.65671740e+04+5.27220927e+04j,  4.36132436e+03+1.74304321e+04j,
          2.48419492e+03+2.48698161e+04j, ...,
          8.75515510e+02+2.71890399e+03j, -1.46111871e+04+2.30462937e+03j,
         -2.71722947e+03-3.01754975e+03j],
        [-2.82050830e+04+7.80500432e+03j, -6.54280601e+03+6.84888286e+03j,
         -1.03099048e+04+8.25676196e+03j, ...,
         -7.43417828e+02+1.19689624e+03j, -4.69189375e+03-5.56714065e+03j,
          6.09640035e+02-1.96284097e+03j],
        [-2.40918400e+04+1.93912971e+04j, -3.44961641e+03+9.36524267e+03j,
         -6.53478939e+03+1.23427784e+04j, ...,
         -2.05189117e+02+1.53500473e+03j, -6.90755854e+03-3.25885871e+03j,
         -2.58394203e+02-2.13480484e+03j],
        ...,
        [-1.83017195e+04-6.24279209e+03j, -6.20125650e+03+1.29916121e+03j,
         -8.70096549e+03+6.51941296e+02j, ...,
         -7.73285739e+02+2.68567085e+02j, -1.75539833e+02-5.00026422e+03j,
          1.14242085e+03-8.37018515e+02j],
        [-1.01082146e+04-1.

In [ ]:
G_R, *_ = gf.sparse_rgf_G_R(0,0)
print(G_R)


[<Compressed Sparse Column sparse matrix of dtype 'complex128'
	with 1600 stored elements and shape (40, 40)>, <Compressed Sparse Column sparse matrix of dtype 'complex128'
	with 1600 stored elements and shape (40, 40)>, <Compressed Sparse Column sparse matrix of dtype 'complex128'
	with 1600 stored elements and shape (40, 40)>, <Compressed Sparse Column sparse matrix of dtype 'complex128'
	with 1600 stored elements and shape (40, 40)>, <Compressed Sparse Column sparse matrix of dtype 'complex128'
	with 1600 stored elements and shape (40, 40)>, <Compressed Sparse Column sparse matrix of dtype 'complex128'
	with 1600 stored elements and shape (40, 40)>, <Compressed Sparse Column sparse matrix of dtype 'complex128'
	with 1600 stored elements and shape (40, 40)>, <Compressed Sparse Column sparse matrix of dtype 'complex128'
	with 1600 stored elements and shape (40, 40)>, <Compressed Sparse Column sparse matrix of dtype 'complex128'
	with 1600 stored elements and shape (40, 40)>, <Compress

In [ ]:
G_R[0].toarray()

array([[-4.16722797e+17+2.29775000e+17j, -1.62501179e+17-3.56423642e+17j,
        -8.16105798e+16-2.67012370e+17j, ...,
        -5.73006477e+17-7.95306967e+17j, -3.42222860e+17+9.91158825e+16j,
        -3.32036302e+17+1.53346063e+16j],
       [ 5.42208279e+18-4.79189459e+17j,  5.37581236e+16+4.48034932e+18j,
        -3.79621306e+17+3.17105891e+18j, ...,
         2.32179451e+18+1.09692253e+19j,  4.03890502e+18+5.43846558e+17j,
         3.54526419e+18+1.37346959e+18j],
       [ 5.90304540e+18-5.84820900e+17j,  1.10338818e+17+4.88172970e+18j,
        -3.77050655e+17+3.45996920e+18j, ...,
         2.65683502e+18+1.19266211e+19j,  4.40757965e+18+5.45985828e+17j,
         3.87923054e+18+1.45574959e+18j],
       ...,
       [-1.05698227e+17+1.37336084e+16j, -4.65439926e+15-8.76145281e+16j,
         4.87699845e+15-6.23464561e+16j, ...,
        -5.42425486e+16-2.12747403e+17j, -7.94584801e+16-7.39401942e+15j,
        -7.04671536e+16-2.40224718e+16j],
       [ 8.51230327e+17-4.34541074e+17j,  3.

In [ ]:
device = Device(channel_length = 9.33e-9)
ham = Hamiltonian(device)
lse = LeadSelfEnergy(device, ham)
gf = GreensFunction(device, ham)

ham.layer_right_lead

0

In [ ]:
Sigma = lse.iterative_self_energy(10,0.0,"right")

In [ ]:
Sigma

array([[ 3.09989380e+00 -2.80322466j,  2.10521006e-01 -1.56051308j,
         2.26741073e+00 -0.49240249j, ...,  2.31476509e+00 -0.33949346j,
        -2.14549888e-03 +0.45193048j, -3.25773622e+00 +0.82163693j],
       [ 2.10521005e-01 -1.56051308j, -1.84260428e+00 -3.14637105j,
         1.21300311e+00 +0.50288087j, ...,  1.37007482e+00 +0.51119148j,
         2.36510256e+00 -0.19424124j, -1.74106082e-01 -0.63727132j],
       [ 2.26741074e+00 -0.49240248j,  1.21300311e+00 +0.50288088j,
        -1.88904988e+00 -2.06917342j, ...,  1.66421465e-01 -2.74538347j,
        -1.64096015e+00 +0.18703294j, -1.29727968e+00 +3.16442548j],
       ...,
       [ 2.31476511e+00 -0.33949346j,  1.37007481e+00 +0.51119148j,
         1.66421471e-01 -2.74538347j, ..., -3.38695193e+00-12.57310668j,
         3.34079161e+00 +2.75043767j,  1.95154684e+00+14.3676834j ],
       [-2.14549528e-03 +0.45193048j,  2.36510256e+00 -0.19424123j,
        -1.64096014e+00 +0.18703294j, ...,  3.34079160e+00 +2.75043767j,
       

In [ ]:
G_R_list, _, _, _, _ = gf.sparse_rgf_G_R(0.1, 0.1)

In [ ]:
G_R_list

[<Compressed Sparse Column sparse matrix of dtype 'complex128'
 	with 19600 stored elements and shape (140, 140)>,
 <Compressed Sparse Column sparse matrix of dtype 'complex128'
 	with 19600 stored elements and shape (140, 140)>,
 <Compressed Sparse Column sparse matrix of dtype 'complex128'
 	with 19600 stored elements and shape (140, 140)>,
 <Compressed Sparse Column sparse matrix of dtype 'complex128'
 	with 19600 stored elements and shape (140, 140)>,
 <Compressed Sparse Column sparse matrix of dtype 'complex128'
 	with 19600 stored elements and shape (140, 140)>,
 <Compressed Sparse Column sparse matrix of dtype 'complex128'
 	with 19600 stored elements and shape (140, 140)>,
 <Compressed Sparse Column sparse matrix of dtype 'complex128'
 	with 19600 stored elements and shape (140, 140)>,
 <Compressed Sparse Column sparse matrix of dtype 'complex128'
 	with 19600 stored elements and shape (140, 140)>,
 <Compressed Sparse Column sparse matrix of dtype 'complex128'
 	with 19600 stor

In [ ]:
sigmaL, sigmaR = gf.self_energy(0.1,0.1)


In [ ]:
sigmaR

array([[-4.24081561+12.01277241j, -3.093276   +6.40853185j,
        -4.93841164-14.37833546j, ...,  0.40950163 -0.82985162j,
         1.80385012 -3.77578518j,  1.60908384 -8.91348787j],
       [-0.87584936+12.67344316j, -2.22562263 +6.86070032j,
        -5.73957339-15.38091501j, ...,  0.13723727 -1.03350802j,
         1.30441544 -3.96700617j, -0.12999812 -9.42804849j],
       [ 3.87859129+10.57766821j,  1.63422628 +5.86655389j,
        -9.08531141-12.93987844j, ...,  0.12400863 -1.23423591j,
         0.41704455 -3.08657622j, -2.04816137 -8.0497393j ],
       ...,
       [-0.70214202 -0.11658832j, -0.46638535 -0.0712991j ,
         0.34938775 -0.15983731j, ..., -1.10812078 -0.09091346j,
         0.72086045 +0.22443978j, -0.53327026 +0.66182993j],
       [ 0.46137096 +0.04071113j,  0.31898938 +0.01912643j,
        -0.19127259 +0.14034258j, ...,  0.70373785 +0.14851621j,
        -0.7064808  -0.17611729j, -0.02788093 -0.51059256j],
       [ 0.38098695 +0.08990997j,  0.24818583 -0.03095915j

In [ ]:
device = Device()
ham = Hamiltonian(device)
lse = LeadSelfEnergy(device, ham)
gf = GreensFunction(device, ham)

G_R, gamma1, gamma2, sigmaL, sigmaR = gf.sparse_rgf_G_R(0.1, 0.1)



In [ ]:
sigmaL.toarray()

array([[ 1.34396467e+01-5.09225615e+00j, -2.09550901e-16-3.92157148e-17j,
         1.59942177e-15-3.49036202e-17j, ...,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j],
       [-2.09550901e-16-3.92157148e-17j,  1.00600013e+01-8.74076972e-02j,
        -1.00996263e+01-8.74076962e-02j, ...,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j],
       [ 1.59942177e-15-3.49036202e-17j, -1.00996263e+01-8.74076962e-02j,
         1.00600013e+01-8.74076972e-02j, ...,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j],
       ...,
       [ 0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j, ...,
        -4.81975000e+00-8.46332501e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j,  0.

In [ ]:
sigma = lse.iterative_self_energy()

In [ ]:
from poisson import PoissonSolver

poissonSolver = PoissonSolver(device)

In [ ]:
poissonSolver.solve_poisson_equation()



In [ ]:
device = Device()
ham = Hamiltonian(device)
rgf = GreensFunction(device, ham)

G_R, gamma1, gamma2, sigmaL, sigmaR= rgf.sparse_rgf_G_R(E=0.1, ky = 0.1)


In [ ]:
print(G_R[0].shape)

In [ ]:
time_start = time.time()
dagger = lambda A: np.conjugate(A.T)
side = "left"
Hpp_matrices = [None] * lead.P
HpP_matrices = [None] * lead.P
hPP,hPP1 = lead.get_layer_hamiltonian(lead.P, side)

HPP = Helper_functions.sparse_inverse(spa.csc_matrix(lead.E * np.eye(hPP.shape[0]) - hPP))
Hpp_matrices[-1], HpP_matrices[-1] = HPP, HPP
for i in range(lead.P - 1, 0, -1):
    
    hpp, hpp1 = lead.get_layer_hamiltonian(i, side)
    Hpp = Helper_functions.sparse_inverse(spa.csc_matrix(lead.E * np.eye(hPP.shape[0]) - \
        hpp - hpp1 @ Hpp_matrices[i] @ dagger(hpp1)))
    Hpp_matrices[i - 1] = Hpp
    HpP = Hpp_matrices[i - 1] @ hpp1 @ HpP_matrices[i]
    HpP_matrices[i - 1] = HpP
    
C22 = Hpp_matrices[1]
C2P = HpP_matrices[1]
C_matrices = [None] * 4
C_matrices[1] = C22
for p in range(3, lead.P + 1):
    hpp, hpp1 = lead.get_layer_hamiltonian(i, side)
    C_matrices[p - 1] = Hpp_matrices[p - 1] + Hpp_matrices[p -1] @ (hpp1 @ C_matrices[p -2] @ dagger(hpp1)) @ Hpp_matrices[p - 1] 

h11, h12 = lead.get_layer_hamiltonian(1, side)

XIs = h11 + h12 @ C_matrices[1] @ dagger(h12)
XI = XIs + dagger(hPP1) @ C_matrices[-1] @ hPP1
PI = h12 @ C2P @ hPP1





In [ ]:
XIs = spa.csc_matrix(XIs)
XI = spa.csc_matrix(XI)
PI = spa.csc_matrix(PI)
I = np.eye(XI.shape[0], dtype=XI)
Z = I * 0
D = lead.E * I.copy() - XI
T = -PI

A = bmat([
    [Z, I],
    [-T.conj().T, -D]
], format='csc')

B = bmat([
    [I, Z],
    [Z, T]
], format='csc')

eigenvalues, eigenvectors = eigs(A, M=B, sigma=1.0, which='LM')




def construct_U_plus_and_Lambda_plus(eigenvalues, eigenvectors, n_dim, epsilon=0.1):
    abs_vals = np.abs(eigenvalues)
    

    is_propagating = np.isclose(abs_vals, 1.0)
    is_evanescent = (abs_vals < 1.0) & (abs_vals > epsilon)
    
    selected_indices = np.where(is_propagating | is_evanescent)[0]
    
    if len(selected_indices) == 0:
        return np.array([], dtype=complex), np.array([],dtype=complex)
        
    filtered_eigenvalues = eigenvalues[selected_indices]
    filtered_eigenvectors = eigenvectors[:, selected_indices]

    Lambda_plus = np.diag(filtered_eigenvalues)
    U_plus = filtered_eigenvectors[:n_dim, :]

    return U_plus, Lambda_plus

U_plus, Lambda = construct_U_plus_and_Lambda_plus(eigenvalues, eigenvectors, T.shape[0], epsilon=0.1)

U_pseudo = np.linalg.pinv(U_plus)
F = U_plus @ Lambda @ U_pseudo

Y = np.linalg.solve(lead.E * I - XIs.toarray() - PI.toarray() @ F, dagger(h12.toarray()))
self_energy = h12 @ Y
time_end = time.time()
print(f"time is: {time_end - time_start}")

In [ ]:
print(self_energy)

In [ ]:
H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=True)
print(H00)

In [ ]:
H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=False)
print(H00)

In [ ]:
def surface_gf(Energy, H00, H10, tol=1e-6): 
    """ 
    This iteratively calculates the surface green's function for the lead based. 
    Although it is tested for 1D, it should be good for 2D surfaces. 
    """

    Energy = Energy
    dagger = lambda A: np.conjugate(A.T)
    
    I = np.eye(H00.shape[0], dtype=complex)
    H01 = dagger(H10)

    epsilon_s = H00.copy()
    epsilon = H00.copy()
    alpha = H01.copy()
    beta = dagger(H10).copy()
    err = 1.0
    first_time = True

    while err > tol:
        if first_time:
            inv_E = Helper_functions.sparse_inverse(spa.csr_matrix(Energy * I) - spa.csr_matrix(epsilon))
            first_time = False
        else:

            inv_E = np.linalg.solve(Energy * I - epsilon, I)
    
        epsilon_s_new = epsilon_s + alpha @ inv_E @ beta
        epsilon_new = epsilon + beta @ inv_E @ alpha + alpha @ inv_E @ beta
        alpha_new = alpha @ inv_E @ alpha
        beta_new = beta @ inv_E @ beta

        err = np.linalg.norm(alpha_new, ord='fro')

        epsilon_s, epsilon, alpha, beta = epsilon_s_new, epsilon_new, alpha_new, beta_new

    return  np.linalg.solve(Energy * I - epsilon_s, I)
H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=True)


surface_gf(0, H00, H10, tol=1e-3)

In [ ]:
def surface_gf(Energy, H00 : np.ndarray, H10: np.ndarray, tol=1e-6): 
    """ 
    This iteratively calculates the surface green's function for the lead based. 
    Although it is tested for 1D, it should be good for 2D surfaces. 
    """
    
    Energy = Energy
    dagger = lambda A: np.conjugate(A.T)
    
    I = np.eye(H00.shape[0], dtype=complex)
    H01 = dagger(H10)

    epsilon_s = H00.copy()
    epsilon = H00.copy()
    alpha = H01.copy()
    beta = dagger(H10).copy()
    err = 1.0

    while err > tol:
        inv_E = np.linalg.solve(Energy * I - epsilon, I)

        epsilon_s_new = epsilon_s + alpha @ inv_E @ beta
        epsilon_new = epsilon + beta @ inv_E @ alpha + alpha @ inv_E @ beta
        alpha_new = alpha @ inv_E @ alpha
        beta_new = beta @ inv_E @ beta

        err = np.linalg.norm(alpha_new, ord='fro')

        epsilon_s, epsilon, alpha, beta = epsilon_s_new, epsilon_new, alpha_new, beta_new

    return  np.linalg.solve(Energy * I - epsilon_s, I)

H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=False)

surface_gf(0, H00, H10)

In [ ]:
from helper import Helper_functions

H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=True)
print(H00)
Helper_functions.sparse_inverse(-H00)
H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=False)
print(H00)
Helper_functions.sparse_inverse(-H00)

In [ ]:
H = device.hamiltonian.create_sparse_hamlitonian(0.1)